In [238]:
!git clone https://github.com/MikolajMezyk/projektML.git
%cd projektML

Cloning into 'projektML'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 2), reused 10 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 233.97 KiB | 4.87 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/projektML/projektML/projektML/projektML/projektML/projektML/projektML/projektML/projektML


In [239]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

###Rozeznanie

In [240]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()




,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [241]:
print(df.shape)

(7043, 21)


In [242]:
print("\nOgólne rozeznanie:")
for column in df.columns:
    if (df[column].dtype == 'object'):
        print(f"Kolumna {column} ma {df[column].unique().shape[0]} różnych wartości")

print("\nCzy są puste wartości?")
display(df.isnull().sum())


Ogólne rozeznanie:
Kolumna customerID ma 7043 różnych wartości
Kolumna gender ma 2 różnych wartości
Kolumna Partner ma 2 różnych wartości
Kolumna Dependents ma 2 różnych wartości
Kolumna PhoneService ma 2 różnych wartości
Kolumna MultipleLines ma 3 różnych wartości
Kolumna InternetService ma 3 różnych wartości
Kolumna OnlineSecurity ma 3 różnych wartości
Kolumna OnlineBackup ma 3 różnych wartości
Kolumna DeviceProtection ma 3 różnych wartości
Kolumna TechSupport ma 3 różnych wartości
Kolumna StreamingTV ma 3 różnych wartości
Kolumna StreamingMovies ma 3 różnych wartości
Kolumna Contract ma 3 różnych wartości
Kolumna PaperlessBilling ma 2 różnych wartości
Kolumna PaymentMethod ma 4 różnych wartości
Kolumna TotalCharges ma 6531 różnych wartości
Kolumna Churn ma 2 różnych wartości

Czy są puste wartości?


,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


### Preprocessing

Id Klienta nie daje na żadnej informacji

In [243]:
df = df.drop(columns=["customerID"])


Konwersja zmiennych numerycznych oraz obsługa brakujących wartości

In [244]:
numeric_cols = ["TotalCharges",  "MonthlyCharges", "tenure"]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # nieprawidłowe wartości -> NaN

print("Ilość Nan:")
print( df[numeric_cols].isna().sum())

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())


Ilość Nan:
TotalCharges      11
MonthlyCharges     0
tenure             0
dtype: int64


Zmiana Churn z Yes/No na 1/0

In [245]:
df["Churn"] = (df["Churn"] == "Yes").astype(int)

Ujednolicenie wartości kategorii zmiennych na Yes i No

In [246]:
replace_map = {
    "No internet service": "No",
    "No phone service": "No"
}

for col in [
    "OnlineSecurity", "OnlineBackup", "DeviceProtection",
    "TechSupport", "StreamingTV", "StreamingMovies", "MultipleLines"
]:
    df[col] = df[col].replace(replace_map)

Podział cech

In [247]:
numeric_features = [
    "tenure",
    "MonthlyCharges",
    "TotalCharges"
]

binary_features = [
    "SeniorCitizen"
]

categorical_features = [
    "gender", "Partner", "Dependents",
    "PhoneService", "MultipleLines",
    "InternetService",
    "OnlineSecurity", "OnlineBackup",
    "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies",
    "Contract", "PaperlessBilling",
    "PaymentMethod"
]


Preprocesor

In [248]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_features)
    ],
    remainder="passthrough"
)


Podział klientów na typy - kubełki

In [249]:
df["tenure_bucket"] = pd.cut(
    df["tenure"],
    bins=[0, 6, 18, 100],
    labels=["0-6", "7-18", "19+"],
    right=True
)

Przygotowywanie danych do modeli

In [250]:
y = df["Churn"]
X = df.drop(columns=["Churn", "tenure_bucket"])

In [251]:
X_train, X_test, y_train, y_test, bucket_train, bucket_test = train_test_split(
    X, y, df["tenure_bucket"],
    test_size=0.2,
    stratify=y,
    random_state=42
)

### Baseline

In [252]:
#najprostszy baseline
df["baseline_pred"] = (
    (df["tenure"] < 6) &
    (df["MonthlyCharges"] > 50)
).astype(int)


In [253]:
def print_classification_report_with_buckets(
    df: pd.DataFrame,
    y_true_col: str,
    y_pred_col: str,
    bucket_col: str = None,
    title: str = None
):
    if title:
        print("=" * len(title))
        print(title)
        print("=" * len(title))

    # -------- GLOBAL --------
    y_true = df[y_true_col]
    y_pred = df[y_pred_col]

    print("\nGLOBAL METRICS")
    print("--------------------")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"Recall   : {recall_score(y_true, y_pred, zero_division=0):.4f}")
    print(f"F1-score : {f1_score(y_true, y_pred, zero_division=0):.4f}")

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

    # -------- BUCKETS --------
    if bucket_col:
        print(f"\nMETRICS PER BUCKET ({bucket_col})")
        print("--------------------")

        bucket_df = (
            df
            .groupby(bucket_col, observed=True)
            .apply(
                lambda x: pd.Series({
                    "recall": recall_score(
                        x[y_true_col],
                        x[y_pred_col],
                        zero_division=0
                    ),
                    "precision": precision_score(
                        x[y_true_col],
                        x[y_pred_col],
                        zero_division=0
                    ),
                    "churn_rate": x[y_true_col].mean(),
                    "n": len(x)
                }),
                include_groups=False
            )
            .reset_index()
        )

        print(bucket_df.to_string(index=False))


In [254]:
print_classification_report_with_buckets(
    df=df,
    y_true_col="Churn",
    y_pred_col="baseline_pred",
    bucket_col="tenure_bucket",
    title="BASELINE MODEL"
)

BASELINE MODEL

GLOBAL METRICS
--------------------
Accuracy : 0.7711
Precision: 0.6680
Recall   : 0.2734
F1-score : 0.3880

Confusion Matrix:
[[4920  254]
 [1358  511]]

METRICS PER BUCKET (tenure_bucket)
--------------------
tenure_bucket   recall  precision  churn_rate      n
          0-6 0.651786   0.672368    0.533333 1470.0
         7-18 0.000000   0.000000    0.343176 1253.0
          19+ 0.000000   0.000000    0.152007 4309.0


### Baseline z baseline fukncji straty LUV

Szacowanie ile klient z nami pozostanie

In [255]:
def expected_remaining_months(tenure):
    if tenure < 6:
        return 24
    elif tenure < 12:
        return 14
    elif tenure < 24:
        return 12
    elif tenure < 48:
        return 10
    else:
        return 6


In [256]:
df["expected_remaining_months"] = df["tenure"].apply(expected_remaining_months)
df["remaining_LTV"] = df["expected_remaining_months"] * df["MonthlyCharges"]


Baseline biznesowej funkcji błędu

In [257]:
def business_loss(y_true, y_pred, remaining_LTV, monthly_fee):
    cost_retention = 20 + 3 * monthly_fee #koszt obsługi potencjalnego churnu w $

    loss = 0

    if y_true == 1 and y_pred == 0:  # FN
        loss = remaining_LTV
    elif y_true == 1 and y_pred == 1:  # TP
        loss = cost_retention
    elif y_true == 0 and y_pred == 1:  # FP
        loss = cost_retention

    return loss

Baseline

In [258]:

df["loss_baseline"] = df.apply(
    lambda row: business_loss(
        y_true=row["Churn"],
        y_pred=row["baseline_pred"],
        remaining_LTV=row["remaining_LTV"],
        monthly_fee=row["MonthlyCharges"]
    ),
    axis=1
)

total_loss_baseline = df["loss_baseline"].sum()

df["loss_no_action"] = df["Churn"] * df["remaining_LTV"]
total_loss_no_action = df["loss_no_action"].sum()

# wynik
print(f"Total loss – no action: ${total_loss_no_action:,.0f}")
print(f"Total loss – baseline: ${total_loss_baseline:,.0f}")
print(f"Loss reduction vs no action: ${(total_loss_no_action - total_loss_baseline):,.0f}")


Total loss – no action: $2,090,674
Total loss – baseline: $1,336,614
Loss reduction vs no action: $754,059


Confusion matrix

In [259]:
df.groupby(["Churn", "baseline_pred"]).size()


Churn  baseline_pred
0      0                4920
       1                 254
1      0                1358
       1                 511
dtype: int64

### Biznesowa funkcja straty

Łączna strata finansowa dla zadanego progu decyzyjnego

In [260]:
def total_business_loss_for_threshold(threshold):
    y_pred_local = (y_proba >= threshold).astype(int)
    df_test_reset = df_test.reset_index(drop=True)

    loss = df_test_reset.apply(
        lambda row: business_loss(
            y_true=row["Churn"],
            y_pred=y_pred_local[row.name],
            remaining_LTV=row["remaining_LTV"],
            monthly_fee=row["MonthlyCharges"]
        ), axis=1
    )
    return loss.sum()

Strata finansowa w kubełkach klientów dla wspólnego progu

In [298]:
def bucket_losses_for_threshold(threshold):
    y_pred_local = (y_proba >= threshold).astype(int)

    df_test_reset = df_test.reset_index(drop=True)

    buckets = df_test_reset["tenure_bucket"].unique()

    results = []
    for b in buckets:
        bucket_df = df_test_reset[df_test_reset["tenure_bucket"] == b]
        loss = bucket_df.apply(
            lambda row: business_loss(
                y_true=row["Churn"],
                y_pred=y_pred_local[row.name],
                remaining_LTV=row["remaining_LTV"],
                monthly_fee=row["MonthlyCharges"]
            ),
            axis=1
        ).sum()
        results.append({
            "bucket": b,
            "loss": loss
        })

    return pd.DataFrame(results)

Strata finansowa w pojedyńczym kubełku klientów dla danego progu

In [261]:
def loss_for_bucket(bucket_df, threshold, y_proba_local):
    #bucket_df_reset = bucket_df.reset_index(drop=True)
    y_pred_local = (y_proba_local >= threshold).astype(int)

    loss = bucket_df.apply(
        lambda row: business_loss(
            y_true=row["Churn"],
            y_pred=y_pred_local[row.name],
            remaining_LTV=row["remaining_LTV"],
            monthly_fee=row["MonthlyCharges"]
        ), axis=1
    )
    return loss.sum()

Wyznaczanie optymalnych progów dla poszczególnych kubełków klientów

In [283]:
def find_optimal_buckets_thresholds(y_proba, df_test):
  buckets = df_test["tenure_bucket"].dropna().unique()
  y_proba_s = pd.Series(y_proba, index=df_test.index)
  thresholds = np.linspace(0, 1, 101)

  results = []
  total_loss_global = 0.0

  for b in buckets:
      bucket_df = df_test[df_test["tenure_bucket"] == b]
      y_proba_local = y_proba_s.loc[bucket_df.index]

      losses = [loss_for_bucket(bucket_df, t, y_proba_local) for t in thresholds]

      best_idx = np.argmin(losses)
      best_threshold = thresholds[best_idx]
      best_loss = losses[best_idx]

      results.append({
          "bucket": b,
          "best_threshold": best_threshold,
          "expected_loss": best_loss
      })

      total_loss_global += best_loss

  results_df = pd.DataFrame(results)


  return results_df, total_loss_global

### Regresja Logistyczna

In [313]:
features = ["tenure", "MonthlyCharges"]  # baseline + możesz dorzucić inne
X = df[features]
y = df["Churn"]

X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(
    X, y, df, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [314]:
clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)

y_proba = clf.predict_proba(X_test_scaled)[:,1]

In [315]:
thresholds = np.linspace(0, 1, 101)
losses = [total_business_loss_for_threshold(t) for t in thresholds]

best_idx = np.argmin(losses)
best_threshold = thresholds[best_idx]

print(f"Best threshold (minimal loss): {best_threshold:.2f}")
print(f"Expected loss at this threshold: ${losses[best_idx]:,.0f}")


Best threshold (minimal loss): 0.21
Expected loss at this threshold: $194,743


In [316]:
bucket_losses_df = bucket_losses_for_threshold(best_threshold)
print(bucket_losses_df)

total_loss = bucket_losses_df["loss"].sum()
print(f"\nTotal expected loss (all buckets): ${total_loss:,.0f}")


  bucket      loss
0    19+  92907.75
1   7-18  44189.75
2    0-6  57223.10
3    NaN      0.00

Total expected loss (all buckets): $194,321


In [317]:
result = find_optimal_buckets_thresholds(y_proba, df_test)
print(result[0])
print(f"\nTotal expected loss (all buckets): ${result[1]:,.0f}")

  bucket  best_threshold  expected_loss
0    19+            0.40       84884.10
1   7-18            0.41       42227.55
2    0-6            0.20       56252.00

Total expected loss (all buckets): $183,364


### RandomForestClassifier

In [310]:
X = df.drop(columns=["Churn", "baseline_pred", "remaining_LTV", "expected_remaining_months"])
X = pd.get_dummies(X, drop_first=True)
y = df["Churn"]

X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(
    X, y, df, test_size=0.3, random_state=42
)

rf = RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42)
rf.fit(X_train, y_train)

y_proba = rf.predict_proba(X_test)[:,1]

thresholds = np.linspace(0, 1, 101)
losses = [total_business_loss_for_threshold(t) for t in thresholds]

best_idx = np.argmin(losses)
best_threshold = thresholds[best_idx]

print(f"Best threshold RF (minimal loss): {best_threshold:.2f}")
print(f"Expected loss RF at this threshold: ${losses[best_idx]:,.0f}")

Best threshold RF (minimal loss): 0.23
Expected loss RF at this threshold: $140,188


In [311]:
bucket_losses_df = bucket_losses_for_threshold(best_threshold)
print(bucket_losses_df)

total_loss = bucket_losses_df["loss"].sum()
print(f"\nTotal expected loss (all buckets): ${total_loss:,.0f}")

  bucket      loss
0    0-6  52852.30
1    19+  54676.90
2   7-18  32658.45
3    NaN      0.00

Total expected loss (all buckets): $140,188


In [312]:
result = find_optimal_buckets_thresholds(y_proba, df_test)
print(result[0])
print(f"\nTotal expected loss (all buckets): ${result[1]:,.0f}")

  bucket  best_threshold  expected_loss
0    0-6            0.23       52852.30
1    19+            0.04       54676.90
2   7-18            0.05       32658.45

Total expected loss (all buckets): $140,188


### GradientBoostingClassifier

In [307]:
X = df.drop(columns=["Churn", "baseline_pred", "expected_remaining_months", "remaining_LTV"])
y = df["Churn"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(
    X, y, df, test_size=0.33, random_state=42, stratify=y
)

scaler = StandardScaler()
numeric_cols = X_train.select_dtypes(include=np.number).columns
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])


gb_clf = GradientBoostingClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    random_state=42
)
gb_clf.fit(X_train_scaled, y_train)

y_proba = gb_clf.predict_proba(X_test_scaled)[:,1]


thresholds = np.linspace(0, 1, 101)
losses = [total_business_loss_for_threshold(t) for t in thresholds]

best_idx = np.argmin(losses)
best_threshold = thresholds[best_idx]

print(f"Best threshold GBM (minimal loss): {best_threshold:.2f}")
print(f"Expected loss GBM at this threshold: ${losses[best_idx]:,.0f}")


Best threshold GBM (minimal loss): 0.01
Expected loss GBM at this threshold: $147,431


In [308]:
bucket_losses_df = bucket_losses_for_threshold(best_threshold)
print(bucket_losses_df)

total_loss = bucket_losses_df["loss"].sum()
print(f"\nTotal expected loss (all buckets): ${total_loss:,.0f}")

  bucket     loss
0    19+  56835.8
1   7-18  33341.9
2    0-6  57253.2
3    NaN      0.0

Total expected loss (all buckets): $147,431


In [309]:
result = find_optimal_buckets_thresholds(y_proba, df_test)
print(result[0])
print(f"\nTotal expected loss (all buckets): ${result[1]:,.0f}")

  bucket  best_threshold  expected_loss
0    19+            0.01        56835.8
1   7-18            0.01        33341.9
2    0-6            0.01        57253.2

Total expected loss (all buckets): $147,431
